# 导入所需的库

In [3]:
import requests
import certifi # 用于验证证书
import pandas as pd
import time

from ratelimit import limits, sleep_and_retry
from requests.exceptions import RequestException

import matplotlib.pyplot as plt
import seaborn as sns

# 1 定义获取函数


In [4]:
# Binance API 限制：5 分钟最多 500 次请求
FIVE_MINUTES = 300  # 5 分钟 = 300 秒
MAX_REQUESTS = 500  # Binance 允许的最大请求数

@sleep_and_retry
@limits(calls=MAX_REQUESTS, period=FIVE_MINUTES)
def get_funding_rate_history_chunk(symbol, start_ts, endpoint_url, limit=1000, verify=True):
    """
    获取最多 limit 条资金费率数据，从 start_ts 开始，带有 API 限流控制和指数退避重试机制。
    """
    params = {
        "symbol": symbol,
        "limit": limit,
        "startTime": start_ts,
    }

    retries = 10 # 最大重试次数
    delay = 2 # 初始等待2s（指数退避）

    for attempt in range(retries):
        try:
            response = requests.get(endpoint_url, params=params, verify=verify)
            response.raise_for_status()  # 如果返回 429、5xx 或其他错误，会抛出异常
            data = response.json()
            return data  # 成功请求则直接返回数据
        
        except RequestException as e:
            print(f"[第 {attempt+1} 次失败] 请求错误：{e}，等待 {delay} 秒后重试...")
            time.sleep(delay)
            delay *= 2  # 指数退避：2s -> 4s -> 8s -> 16s -> ...

    # 如果循环结束还没成功，抛出异常
    raise Exception(f"连续 {retries} 次请求失败，终止请求。")

def get_full_funding_rate_history(symbol, endpoint_url="https://fapi.binance.com/fapi/v1/fundingRate", verify=True):
    """
    以 `limit=1000` 条为单位进行翻页，获取完整的资金费率数据。
    """
    all_data = []
    
    # 可以从交易所上线日期、或只从 2020年 开始；如果想拿尽量早，可以写死一个日期
    start_ts = 1586908800000  # 这里是 2020-04-15 00:00:00 (ms)
    now_ts = int(time.time() * 1000)
    
    # 用 while 循环不断地往后翻，直到超过当前时间
    while start_ts < now_ts:
        data_part = get_funding_rate_history_chunk(symbol, start_ts, endpoint_url, limit=1000, verify=verify)

        if not data_part:
            break # 如果没有数据返回，说明到头了
    
        # 合并到 all_data
        all_data.extend(data_part)
        
        # 如果返回的数据不足 1000 条，说明没有更多数据
        if len(data_part) < 1000:
            break

        # 获取最后一条数据的时间戳，并 +1，作为下一次查询的起始时间
        start_ts = data_part[-1]["fundingTime"] + 1
    
    return all_data

# 2 获取所有币对信息， 并遍历所有币对，采集资金费率历史数据

In [5]:
exchange_info_url = "https://fapi.binance.com/fapi/v1/exchangeInfo"
try:
    response = requests.get(exchange_info_url, verify=certifi.where())
except Exception as e:
    print(f"获取币对信息时发生异常：{e}")
    response = None

if response is None or response.status_code != 200:
    raise Exception("无法获取币对信息，请检查网络或 API 状态。")

exchange_data = response.json()
# 筛选出状态为 TRADING 且合约类型为 PERPETUAL 的币对
symbols = [s["symbol"] for s in exchange_data["symbols"]
           if s.get("contractType") == "PERPETUAL" and s.get("status") == "TRADING"]

print(f"共获取到 {len(symbols)} 个币对：")
print(symbols)

共获取到 386 个币对：
['BTCUSDT', 'ETHUSDT', 'BCHUSDT', 'XRPUSDT', 'EOSUSDT', 'LTCUSDT', 'TRXUSDT', 'ETCUSDT', 'LINKUSDT', 'XLMUSDT', 'ADAUSDT', 'XMRUSDT', 'DASHUSDT', 'ZECUSDT', 'XTZUSDT', 'BNBUSDT', 'ATOMUSDT', 'ONTUSDT', 'IOTAUSDT', 'BATUSDT', 'VETUSDT', 'NEOUSDT', 'QTUMUSDT', 'IOSTUSDT', 'THETAUSDT', 'ALGOUSDT', 'ZILUSDT', 'KNCUSDT', 'ZRXUSDT', 'COMPUSDT', 'DOGEUSDT', 'SXPUSDT', 'KAVAUSDT', 'BANDUSDT', 'RLCUSDT', 'MKRUSDT', 'SNXUSDT', 'DOTUSDT', 'DEFIUSDT', 'YFIUSDT', 'BALUSDT', 'CRVUSDT', 'TRBUSDT', 'RUNEUSDT', 'SUSHIUSDT', 'EGLDUSDT', 'SOLUSDT', 'ICXUSDT', 'STORJUSDT', 'UNIUSDT', 'AVAXUSDT', 'ENJUSDT', 'FLMUSDT', 'KSMUSDT', 'NEARUSDT', 'AAVEUSDT', 'FILUSDT', 'RSRUSDT', 'LRCUSDT', 'BELUSDT', 'AXSUSDT', 'ALPHAUSDT', 'ZENUSDT', 'SKLUSDT', 'GRTUSDT', '1INCHUSDT', 'CHZUSDT', 'SANDUSDT', 'ANKRUSDT', 'RVNUSDT', 'SFPUSDT', 'COTIUSDT', 'CHRUSDT', 'MANAUSDT', 'ALICEUSDT', 'HBARUSDT', 'ONEUSDT', 'LINAUSDT', 'STMXUSDT', 'DENTUSDT', 'CELRUSDT', 'HOTUSDT', 'MTLUSDT', 'OGNUSDT', 'NKNUSDT', '1000SHIBUSD

In [6]:
all_funding_data = []
for symbol in symbols:
    print(f"\n正在采集 {symbol} 的全历史资金费率数据...")
    funding_data = get_full_funding_rate_history(symbol, verify=certifi.where())
    print(f"【{symbol}】获取到 {len(funding_data)} 条记录")
    
    # 为每条记录增加币对标识
    for record in funding_data:
        record["symbol"] = symbol
    all_funding_data.extend(funding_data)
    
    # 为避免请求过快，每个币对之间暂停 0.5 秒
    time.sleep(0.5)


正在采集 BTCUSDT 的全历史资金费率数据...
【BTCUSDT】获取到 5290 条记录

正在采集 ETHUSDT 的全历史资金费率数据...
【ETHUSDT】获取到 5290 条记录

正在采集 BCHUSDT 的全历史资金费率数据...
【BCHUSDT】获取到 5290 条记录

正在采集 XRPUSDT 的全历史资金费率数据...
【XRPUSDT】获取到 5290 条记录

正在采集 EOSUSDT 的全历史资金费率数据...
【EOSUSDT】获取到 5290 条记录

正在采集 LTCUSDT 的全历史资金费率数据...
【LTCUSDT】获取到 5290 条记录

正在采集 TRXUSDT 的全历史资金费率数据...
【TRXUSDT】获取到 5290 条记录

正在采集 ETCUSDT 的全历史资金费率数据...
【ETCUSDT】获取到 5290 条记录

正在采集 LINKUSDT 的全历史资金费率数据...
【LINKUSDT】获取到 5290 条记录

正在采集 XLMUSDT 的全历史资金费率数据...
【XLMUSDT】获取到 5290 条记录

正在采集 ADAUSDT 的全历史资金费率数据...
【ADAUSDT】获取到 5290 条记录

正在采集 XMRUSDT 的全历史资金费率数据...
【XMRUSDT】获取到 5290 条记录

正在采集 DASHUSDT 的全历史资金费率数据...
[第 1 次失败] 请求错误：HTTPSConnectionPool(host='fapi.binance.com', port=443): Max retries exceeded with url: /fapi/v1/fundingRate?symbol=DASHUSDT&limit=1000&startTime=1730880000001 (Caused by SSLError(SSLEOFError(8, '[SSL: UNEXPECTED_EOF_WHILE_READING] EOF occurred in violation of protocol (_ssl.c:1000)')))，等待 2 秒后重试...
【DASHUSDT】获取到 5290 条记录

正在采集 ZECUSDT 的全历史资金费率数据...
【ZE

# 3 将数据保存为 CSV 文件，方便后续在 Notebook 中加载分析

In [7]:
df = pd.DataFrame(all_funding_data)
csv_filename = "binance_funding_rate_history_full.csv"
df.to_csv(csv_filename, index=False)
print(f"\n所有数据已保存到文件：{csv_filename}")


所有数据已保存到文件：binance_funding_rate_history_full.csv
